In [ ]:
# Importing necessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Reading Applications Raw Data
df=pd.read_csv('Code1_application.csv')

In [ ]:
# Cleaning data
df['address']=np.where(df['address']=='123 MAIN ST',df['record'],df['address'])
df['homephone']=np.where(df['homephone']==9999999999, df['record'], df['homephone'])
df['ssn'] = np.where(df['ssn']==999999999, df['record'], df['ssn'])
df['dob'] = np.where(df['dob']==19070626, df['record'], df['dob'])

In [ ]:
# Creating Same lengths for variables
df['homephone'] = df['homephone'].astype(str).apply(lambda x: x.zfill(10))
df['zip5'] = df['zip5'].astype(str).apply(lambda x: x.zfill(5))
df['ssn'] = df['ssn'].astype(str).apply(lambda x: x.zfill(9))
df['dob'] = df['dob'].astype(str).apply(lambda x: x.zfill(8))

In [ ]:
# Converting to correct format
df = df.astype(str)
df['record'] = df['record'].astype(int)
df['fraud_label'] = df['fraud_label'].astype(int)
df['date'] = pd.to_datetime(df['date'])

## Base Variable Creation

In [ ]:
# creating 16 new base variables along with ssn and homephone
df['name']= df['firstname']+df['lastname']
df['namedob']=df['firstname']+df['lastname']+df['dob']
df['fulladdress']=df['address']+df['zip5']

df['ssnfulladdress']=df['ssn']+df['fulladdress']
df['ssnnamedob']=df['ssn']+df['namedob']
df['ssnphone']=df['ssn']+df['homephone']

df['namedobfulladdress']=df['namedob']+df['fulladdress']
df['fulladdressphone']=df['fulladdress']+df['homephone']
df['namedobphone']=df['namedob']+df['homephone']
df['firstnamessn']=df['firstname']+df['ssn']
df['lastnamessn']=df['lastname']+df['ssn']

df['namephone']=df['name']+df['homephone']
df['namefulladdress']=df['name']+df['fulladdress']
df['namezip']=df['name']+df['zip5']
df['namephonefulladdress']=df['name']+df['homephone']+df['fulladdress']
df['namessn']=df['name']+df['ssn']

base_variables = ['ssn','homephone','name','namedob','fulladdress','ssnfulladdress',
                 'ssnnamedob','ssnphone','namedobfulladdress','fulladdressphone','namedobphone', 'firstnamessn',
                 'lastnamessn','namephone','namefulladdress','namezip','namephonefulladdress','namessn']


In [ ]:
# Set date as index for rolling count
df = df.set_index('date')

## Velocity Logic

In [ ]:
# calculates all the velocity variables
for var in base_variables:
        df['vel_{}_1'.format(var)] = getattr(df.groupby(var)['record'].rolling('1D'),'count')().values-1
        df['vel_{}_3'.format(var)] = getattr(df.groupby(var)['record'].rolling('3D'),'count')().values-1
        df['vel_{}_7'.format(var)] = getattr(df.groupby(var)['record'].rolling('7D'),'count')().values-1
        df['vel_{}_14'.format(var)] = getattr(df.groupby(var)['record'].rolling('14D'),'count')().values-1
        df['vel_{}_30'.format(var)] = getattr(df.groupby(var)['record'].rolling('30D'),'count')().values-1
        df['vel_{}_90'.format(var)] = getattr(df.groupby(var)['record'].rolling('90D'),'count')().values-1
        df['vel_{}_180'.format(var)] = getattr(df.groupby(var)['record'].rolling('180D'),'count')().values-1
        

## Relative Velocity Logic

In [ ]:
# calculates relative velocity variables using 1 as base
for var in base_variables:
    for num in [3,7,14,30,90,180]:
        df['relv_{}_{}'.format(var,num)] = df['vel_{}_1'.format(var)]/ df['vel_{}_{}'.format(var,num)]
        
#replacing nan with 0
df.replace(np.nan, 0, inplace=True)
df = df.reset_index()

## dayssince logic

In [ ]:
#Days Since Variable Creation

base_date = pd.to_datetime('2016-01-01')

for i in base_variables:
    variable_name='dayssince_{}'.format(i)
    df[variable_name] = df.groupby(i)['date'].diff().dt.days
    df['temp'] = (df.date - base_date)
    df['temp'] = df.apply(lambda x: x.temp.days, axis=1)
    df['temp'] = df['temp'].astype(float)
    df['temp'][df['temp'] <20] = 19
    df[variable_name] = df[variable_name].replace(np.nan,df['temp']+1)
    df = df.drop('temp',axis = 1)


In [ ]:
# Subtracts days since variable from max to keep it consistent (higher number means bad and lower number means good now)
#df.info()

days_since_columns = ['dayssince_ssn','dayssince_homephone','dayssince_name','dayssince_namedob','dayssince_fulladdress','dayssince_ssnfulladdress',
 'dayssince_ssnnamedob','dayssince_ssnphone','dayssince_namedobfulladdress','dayssince_fulladdressphone','dayssince_namedobphone',
 'dayssince_firstnamessn','dayssince_lastnamessn','dayssince_namephone','dayssince_namefulladdress','dayssince_namezip',
 'dayssince_namephonefulladdress','dayssince_namessn']


for i in days_since_columns: 
    maximum = df[i].max()
    df[i] = maximum - df[i]


In [ ]:
# Creating day of week target encoding
df['dayofweek'] = df['date'].dt.day_name()
df['risk_dayofweek'] = df.groupby('dayofweek')['fraud_label'].transform('mean')

In [ ]:
# Dropping original variables
original_variables = ['date','ssn','firstname','lastname','address','zip5','dob','homephone',
                      'name','namedob','fulladdress','ssnfulladdress','ssnnamedob','ssnphone','namedobfulladdress',
                      'fulladdressphone','namedobphone','firstnamessn','lastnamessn','namephone','namefulladdress',
                      'namezip','namephonefulladdress','namessn','dayofweek']

df.drop(original_variables,axis = 1,inplace = True)

In [ ]:
# moving fraud_label to last

df['fraud_label1'] = df['fraud_label']*1
df.drop('fraud_label',axis = 1,inplace = True)
df.rename(columns={'fraud_label1': 'fraud_label'},inplace = True)

In [ ]:
pd.set_option('display.max_columns', None)  
df.head()

In [ ]:
df.to_csv("All Variable Data.csv")